In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output
import uuid
clear_output(wait=True)
# Function to create and set up a widget with its custom JavaScript element
slider_uuid = "uuid"+str(uuid.uuid4()).replace('-','')
slider_widget = widgets.IntSlider(value=30, min=0, max=100)
slider_widget.add_class(slider_uuid)
text_uuid = "uuid"+str(uuid.uuid4()).replace('-','')
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)
output1=widgets.Output();
js_code=f"""
    const canvas = document.createElement("canvas");
    canvas.style.display="inline-block";
    canvas.style.width="400px";
    canvas.style.height="400px";
    //element.appendChild(canvas);
    const slider= document.createElement('input');  
    slider.style.width="200px";
    slider.type="range";
    slider.min=0;
    slider.max=100;
    slider.value=40;
    element.appendChild(slider);
    const sliderWidget=document.querySelector(".{slider_uuid}");
    const text= document.createElement('input');  
    text.style.width="200px";
    text.type="text";
    text.value="initial text2";
    element.appendChild(text);
    const textWidget=document.querySelector(".{text_uuid}"); 
    alert("element= "+element+" {slider_uuid= }"+"sliderWidget= "+sliderWidget+" textWidget="+textWidget );
    """
with output1:
    display(Javascript(js_code))
print(js_code)

In [ ]:
display(slider_widget,text_widget,output1)

In [2]:
def capture_console(code):
    return (
"""
try{console._o=console._o||{log:console.log,error:console.error,warn:console.warn,info:console.info};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{console[t]=console._o[t];let d=element.consoleOutput;if(!d){d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){return n.apply(this,a);}:null;o=function(...a){(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${t.toUpperCase()}] ${s}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${c};`;d.appendChild(m);};console[t]=o;});
"""  +
code + 
"""
}catch(e){console.error("Error:",e);}finally{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}
"""
)

In [ ]:
# Query the DOM
def query_dom():
    js_code = """
    const elements = document.querySelectorAll('.jupyter-widgets');
    const results = [];
    elements.forEach((el, index) => {
        const input = el.querySelector('input');
        const classes = Array.from(el.classList);
        const inputValue = input ? input.value : 'No input found';
        results.push({
            index: index,
            name: el.name??"",
            id: el.id??"",
            classes: classes,
            inputValue: inputValue,
            style: el.style.display || 'Not set'
        });
    });
    console.log('DOM Query Results:', results);
    """
    display(Javascript(capture_console(js_code)))

query_dom()

In [ ]:
display(output1)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid"+str(uuid.uuid4()).replace('-','')
text_uuid = "uuid"+str(uuid.uuid4()).replace('-','')

# Create the slider and text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)

# Create the Output widget
output1 = widgets.Output()

# JavaScript code to investigate the slider
js_code = f"""
console.log("Investigating slider widget...");
const sliderWidget = document.querySelector('.{slider_uuid}');
if (sliderWidget) {{
    console.log("Slider innerHTML:", sliderWidget.innerHTML);
    console.log("Label:", sliderWidget.querySelector('.widget-label')?.textContent || "Not found");
    console.log("Hidden input:", sliderWidget.querySelector('input[type="hidden"]')?.value || "Not found");
    console.log("noUiSlider base:", sliderWidget.querySelector('.noUi-base') || "Not found");
    console.log("noUiSlider handle:", sliderWidget.querySelector('.noUi-handle') || "Not found");
    console.log("Data attributes:", sliderWidget.dataset);

    // Attempt to update the value if we find a suitable element
    const hiddenInput = sliderWidget.querySelector('input[type="hidden"]');
    const handle = sliderWidget.querySelector('.noUi-handle');
    if (hiddenInput) {{
        console.log("Updating hidden input...");
        hiddenInput.value = "75";
        const changeEvent = new Event('change', {{ bubbles: true }});
        hiddenInput.dispatchEvent(changeEvent);
        const inputEvent = new Event('input', {{ bubbles: true }});
        hiddenInput.dispatchEvent(inputEvent);
    }} else if (handle) {{
        console.log("Updating noUiSlider handle...");
        handle.style.left = "75%"; // Assuming min=0, max=100
        const changeEvent = new Event('change', {{ bubbles: true }});
        handle.dispatchEvent(changeEvent);
        const inputEvent = new Event('input', {{ bubbles: true }});
        handle.dispatchEvent(inputEvent);
        const slideEvent = new Event('slide', {{ bubbles: true }});
        handle.dispatchEvent(slideEvent);
    }} else {{
        console.log("No suitable element found to update the slider value.");
    }}
}} else {{
    console.log("Slider widget not found.");
}}
"""

# Display the JavaScript in the Output widget
with output1:
    display(Javascript(capture_console(js_code)))

# Display the widgets
display(slider, text_widget, output1)

# Print the slider's value to verify the update
print("Slider value after JavaScript:", slider.value)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')

# Create the slider and text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)

# Create the Output widget
output1 = widgets.Output()

# JavaScript code to simulate user interaction
js_code = f"""
console.log("Investigating slider widget...");
const sliderWidget = document.querySelector('.{slider_uuid}');
if (sliderWidget) {{
    console.log("Slider innerHTML:", sliderWidget.innerHTML);
    console.log("Label:", sliderWidget.querySelector('.widget-label')?.textContent || "Not found");
    console.log("Hidden input:", sliderWidget.querySelector('input[type="hidden"]')?.value || "Not found");
    console.log("noUiSlider base:", sliderWidget.querySelector('.noUi-base') || "Not found");
    console.log("noUiSlider handle:", sliderWidget.querySelector('.noUi-handle') || "Not found");
    console.log("Data attributes:", sliderWidget.dataset);

    // Simulate user interaction to set the slider to 75
    const handle = sliderWidget.querySelector('.noUi-handle');
    if (handle) {{
        console.log("Simulating user interaction...");
        // Start dragging
        const mousedownEvent = new MouseEvent('mousedown', {{ bubbles: true }});
        handle.dispatchEvent(mousedownEvent);

        // Move to 75% (calculate x-coordinate based on slider width)
        const sliderBase = sliderWidget.querySelector('.noUi-base');
        const rect = sliderBase.getBoundingClientRect();
        const xPosition = rect.left + (rect.width * 0.75); // 75% of the slider's width
        const mousemoveEvent = new MouseEvent('mousemove', {{
            bubbles: true,
            clientX: xPosition
        }});
        document.dispatchEvent(mousemoveEvent);

        // End dragging
        const mouseupEvent = new MouseEvent('mouseup', {{ bubbles: true }});
        document.dispatchEvent(mouseupEvent);

        // Dispatch additional events
        const slideEvent = new Event('slide', {{ bubbles: true }});
        sliderBase.dispatchEvent(slideEvent);
        const updateEvent = new Event('update', {{ bubbles: true }});
        sliderBase.dispatchEvent(updateEvent);
    }} else {{
        console.log("No noUiSlider handle found.");
    }}
}} else {{
    console.log("Slider widget not found.");
}}
"""

# Display the JavaScript in the Output widget
with output1:
    display(Javascript(capture_console(js_code)))

# Display the widgets
display(slider, text_widget, output1)

# Print the slider's value to verify the update
print("Slider value after JavaScript:", slider.value)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')

# Create the slider and text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)

# Create the Output widget
output1 = widgets.Output()

# JavaScript code to simulate user interaction
js_code = f"""
console.log("Investigating slider widget...");
const sliderWidget = document.querySelector('.{slider_uuid}');
if (sliderWidget) {{
    console.log("Slider innerHTML:", sliderWidget.innerHTML);
    console.log("Label:", sliderWidget.querySelector('.widget-label')?.textContent || "Not found");
    console.log("Hidden input:", sliderWidget.querySelector('input[type="hidden"]')?.value || "Not found");
    console.log("noUiSlider base:", sliderWidget.querySelector('.noUi-base') || "Not found");
    console.log("noUiSlider handle:", sliderWidget.querySelector('.noUi-handle') || sliderWidget.querySelector('.noUi-handle-lower') || "Not found");
    console.log("Data attributes:", sliderWidget.dataset);

    // Simulate user interaction with a delay
    setTimeout(() => {{
try{{console._o=console._o||{{log:console.log,error:console.error,warn:console.warn,info:console.info}};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{{console[t]=console._o[t];let d=element.consoleOutput;if(!d){{d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){{return n.apply(this,a);}}:null;o=function(...a){{(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${{t.toUpperCase()}}] ${{s}}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${{c}};`;d.appendChild(m);}};console[t]=o;}});
console.log("----- Inside timeout callback function! -----");
        const handle = sliderWidget.querySelector('.noUi-handle') || sliderWidget.querySelector('.noUi-handle-lower');
        if (handle) {{
            console.log("Simulating user interaction...");
            const rect = handle.getBoundingClientRect();
            const yPosition = rect.top + (rect.height / 2);

            // Start dragging
            const mousedownEvent = new MouseEvent('mousedown', {{
                bubbles: true,
                clientX: rect.left,
                clientY: yPosition
            }});
            handle.dispatchEvent(mousedownEvent);

            // Move to 75%
            const sliderBase = sliderWidget.querySelector('.noUi-base');
            const baseRect = sliderBase.getBoundingClientRect();
            const xPosition = baseRect.left + (baseRect.width * 0.75);
            const mousemoveEvent = new MouseEvent('mousemove', {{
                bubbles: true,
                clientX: xPosition,
                clientY: yPosition
            }});
            document.dispatchEvent(mousemoveEvent);

            // End dragging
            const mouseupEvent = new MouseEvent('mouseup', {{
                bubbles: true,
                clientX: xPosition,
                clientY: yPosition
            }});
            document.dispatchEvent(mouseupEvent);

            // Additional events on sliderBase and sliderWidget
            const slideEvent = new Event('slide', {{ bubbles: true }});
            sliderBase.dispatchEvent(slideEvent);
            const updateEvent = new Event('update', {{ bubbles: true }});
            sliderBase.dispatchEvent(updateEvent);

            const changeEvent = new Event('change', {{ bubbles: true }});
            sliderBase.dispatchEvent(changeEvent);
            sliderWidget.dispatchEvent(changeEvent);

            const inputEvent = new Event('input', {{ bubbles: true }});
            sliderBase.dispatchEvent(inputEvent);
            sliderWidget.dispatchEvent(inputEvent);
        }} else {{
            console.log("No noUiSlider handle found after delay.");
        }}
}}catch(e){{console.error("Error:",e);}}finally{{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}}


    }}, 1000);
}} else {{
    console.log("Slider widget not found.");
}}
"""

# Display the JavaScript in the Output widget
with output1:
    display(Javascript(capture_console(js_code)))

# Display the widgets
display(slider, text_widget, output1)

# Print the slider's value to verify the update
print("Slider value after JavaScript:", slider.value)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
hidden_text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')

# Create the slider, text, and hidden text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)
hidden_text = widgets.Text(value="50", layout={'display': 'none'})
hidden_text.add_class(hidden_text_uuid)

# Link the slider to the hidden text widget
slider.observe(lambda change: setattr(hidden_text, 'value', str(change['new'])), names='value')

# Create the Output widget
output1 = widgets.Output()

# JavaScript code to try event simulation and use hidden text as a fallback
js_code = f"""
console.log("Investigating slider widget...");
const sliderWidget = document.querySelector('.{slider_uuid}');
if (sliderWidget) {{
    console.log("Slider innerHTML:", sliderWidget.innerHTML);
    console.log("Readout:", sliderWidget.querySelector('.widget-readout')?.textContent || "Not found");
    console.log("Hidden input:", sliderWidget.querySelector('input[type="hidden"]')?.value || "Not found");
    console.log("noUiSlider base:", sliderWidget.querySelector('.noUi-base') || "Not found");
    console.log("noUiSlider handle:", sliderWidget.querySelector('.noUi-handle') || sliderWidget.querySelector('.noUi-handle-lower') || "Not found");
    console.log("Data attributes:", sliderWidget.dataset);

    // Simulate user interaction
    setTimeout(() => {{
    try{{console._o=console._o||{{log:console.log,error:console.error,warn:console.warn,info:console.info}};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{{console[t]=console._o[t];let d=element.consoleOutput;if(!d){{d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){{return n.apply(this,a);}}:null;o=function(...a){{(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${{t.toUpperCase()}}] ${{s}}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${{c}};`;d.appendChild(m);}};console[t]=o;}});

        const handle = sliderWidget.querySelector('.noUi-handle') || sliderWidget.querySelector('.noUi-handle-lower');
        if (handle) {{
            console.log("Simulating user interaction...");
            const rect = handle.getBoundingClientRect();
            const yPosition = rect.top + (rect.height / 2);

            // Try touch events
            const touchStartEvent = new Event('touchstart', {{ bubbles: true }});
            touchStartEvent.touches = [{{ clientX: rect.left, clientY: yPosition }}];
            handle.dispatchEvent(touchStartEvent);

            const sliderBase = sliderWidget.querySelector('.noUi-base');
            const baseRect = sliderBase.getBoundingClientRect();
            const startX = baseRect.left;
            const endX = baseRect.left + (baseRect.width * 0.75);
            const steps = 10;
            for (let i = 0; i <= steps; i++) {{
                const xPosition = startX + ((endX - startX) * (i / steps));
                const touchMoveEvent = new Event('touchmove', {{ bubbles: true }});
                touchMoveEvent.touches = [{{ clientX: xPosition, clientY: yPosition }}];
                document.dispatchEvent(touchMoveEvent);
            }}

            const touchEndEvent = new Event('touchend', {{ bubbles: true }});
            touchEndEvent.changedTouches = [{{ clientX: endX, clientY: yPosition }}];
            document.dispatchEvent(touchEndEvent);

            // Fallback to mouse events
            const mousedownEvent = new MouseEvent('mousedown', {{
                bubbles: true,
                clientX: rect.left,
                clientY: yPosition,
                buttons: 1
            }});
            handle.dispatchEvent(mousedownEvent);

            for (let i = 0; i <= steps; i++) {{
                const xPosition = startX + ((endX - startX) * (i / steps));
                const mousemoveEvent = new MouseEvent('mousemove', {{
                    bubbles: true,
                    clientX: xPosition,
                    clientY: yPosition,
                    buttons: 1
                }});
                document.dispatchEvent(mousemoveEvent);
            }}

            const mouseupEvent = new MouseEvent('mouseup', {{
                bubbles: true,
                clientX: endX,
                clientY: yPosition
            }});
            document.dispatchEvent(mouseupEvent);

            // Additional events
            const slideEvent = new Event('slide', {{ bubbles: true }});
            sliderBase.dispatchEvent(slideEvent);
            const updateEvent = new Event('update', {{ bubbles: true }});
            sliderBase.dispatchEvent(updateEvent);
            const changeEvent = new Event('change', {{ bubbles: true }});
            sliderBase.dispatchEvent(changeEvent);
            sliderWidget.dispatchEvent(changeEvent);
            const inputEvent = new Event('input', {{ bubbles: true }});
            sliderBase.dispatchEvent(inputEvent);
            sliderWidget.dispatchEvent(inputEvent);
        }} else {{
            console.log("No noUiSlider handle found after delay.");
        }}

        // Fallback: Update the hidden text widget to set the slider to 75
        console.log("Attempting to update via hidden text widget...");
        const hiddenTextWidget = document.querySelector('.{hidden_text_uuid}');
        if (hiddenTextWidget) {{
            const input = hiddenTextWidget.querySelector('input');
            if (input) {{
                input.value = "75";
                const changeEvent = new Event('change', {{ bubbles: true }});
                input.dispatchEvent(changeEvent);
                console.log("changeEvent dispatched!");
                const inputEvent = new Event('input', {{ bubbles: true }});
                input.dispatchEvent(inputEvent);
                console.log("inputEvent dispatched!");
            }} else {{
                console.log("Hidden text input not found.");
            }}
        }} else {{
            console.log("Hidden text widget not found.");
        }}
        
        }}catch(e){{console.error("Error:",e);}}finally{{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}}

    }}, 1000);
}} else {{
    console.log("Slider widget not found.");
}}
"""

# Display the JavaScript in the Output widget
with output1:
    display(Javascript(capture_console(js_code)))

# Display the widgets
display(slider, text_widget, hidden_text, output1)

# Print the slider's value to verify the update
print("Slider value after JavaScript:", slider.value)

In [4]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import uuid

# Create unique UUIDs for the widgets
slider_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')
hidden_text_uuid = "uuid" + str(uuid.uuid4()).replace('-', '')

# Create the slider, text, and hidden text widgets with unique classes
slider = widgets.IntSlider(value=50, min=0, max=100)
slider.add_class(slider_uuid)
text_widget = widgets.Text(value="initial text")
text_widget.add_class(text_uuid)
hidden_text = widgets.Text(value="50", layout={'display': 'none'})
hidden_text.add_class(hidden_text_uuid)

# Two-way linking
slider.observe(lambda change: setattr(hidden_text, 'value', str(change['new'])), names='value')
hidden_text.observe(lambda change: setattr(slider, 'value', int(change['new'])), names='value')

# Create the Output widget with custom inputs
output1 = widgets.Output()
with output1:
    display(Javascript(capture_console(f"""
    const sliderInput = document.createElement('input');
    sliderInput.type = 'range';
    sliderInput.min = 0;
    sliderInput.max = 100;
    sliderInput.value = 50;
    sliderInput.style.width = '210px';
    element.appendChild(sliderInput);

    const textInput = document.createElement('input');
    textInput.type = 'text';
    textInput.value = 'initial text';
    textInput.style.width = '200px';
    element.appendChild(textInput);

    // JavaScript-to-Python synchronization
    const sliderWidget = document.querySelector('.{slider_uuid}');
    const hiddenTextWidget = document.querySelector('.{hidden_text_uuid}');
    const textWidget = document.querySelector('.{text_uuid}');

    sliderInput.addEventListener('input', () => {{
        if (hiddenTextWidget) {{
            const input = hiddenTextWidget.querySelector('input');
            if (input) {{
                input.value = sliderInput.value;
                const changeEvent = new Event('change', {{ bubbles: true }});
                input.dispatchEvent(changeEvent);
                const inputEvent = new Event('input', {{ bubbles: true }});
                input.dispatchEvent(inputEvent);
            }}
        }}
    }});

    textInput.addEventListener('input', () => {{
        if (textWidget) {{
            const input = textWidget.querySelector('input');
            if (input) {{
                input.value = textInput.value;
                const changeEvent = new Event('change', {{ bubbles: true }});
                input.dispatchEvent(changeEvent);
                const inputEvent = new Event('input', {{ bubbles: true }});
                input.dispatchEvent(inputEvent);
            }}
        }}
    }});

    // Python-to-JavaScript synchronization (polling)
    setInterval(() => {{
    try{{console._o=console._o||{{log:console.log,error:console.error,warn:console.warn,info:console.info}};[['log'],['error','red'],['warn'],['info']].forEach(([t,c="black"])=>{{console[t]=console._o[t];let d=element.consoleOutput;if(!d){{d=document.createElement("div");d.id="console-output";d.style.cssText="white-space:pre-wrap;font-family:monospace;padding:0px;background:#f0f0f0;line-height:1.1;";element.appendChild(d);element.consoleOutput=d;}}let o=console[t],n=o;while(n&&n.toString().indexOf("[native code]")<0)n=n.apply?function(...a){{return n.apply(this,a);}}:null;o=function(...a){{(n||console._o[t]).apply(console,a);let s=a.map(e=>typeof e==='object'?JSON.stringify(e,null,2):e).join(' ');const m=document.createElement("div");m.innerText=`[${{t.toUpperCase()}}] ${{s}}`;m.style.cssText=`margin:0;line-height:1.1;padding:0px 0;color:${{c}};`;d.appendChild(m);}};console[t]=o;}});
  //       console.log("in set_imtervall");
        if (sliderWidget) {{
            const readout = sliderWidget.querySelector('.widget-readout');
            if (readout && sliderInput.value !== readout.textContent) {{
                sliderInput.value = readout.textContent;
            }}
        }}
        if (textWidget) {{
            const input = textWidget.querySelector('input');
            if (input && textInput.value !== input.value) {{
                textInput.value = input.value;
            }}
        }}
    }}catch(e){{console.error("Error:",e);}}finally{{[['log'],['error','red'],['warn'],['info']].forEach(([t])=>console._o&&console._o[t]&&(console[t]=console._o[t]));}}
    }}, 100); // Poll every 100ms
    """)))

# Display the widgets
display(slider, text_widget, hidden_text, output1)

IntSlider(value=50, _dom_classes=('uuid5a7174822f3747cc918bc71281b81042',))

Text(value='initial text', _dom_classes=('uuid00cc23fbcf224806af983dfe556ffd2a',))

Text(value='50', layout=Layout(display='none'), _dom_classes=('uuid50c1eb9224fa4f448ffa3fc9dba6a048',))

Output()

In [ ]:
print(capture_console('').replace('{','{{').replace('}','}}'))


In [ ]:
slider.value